# <hr style="clear: both" />

# Running Spark in YARN-client mode

This notebook demonstrates how to set up a SparkContext that uses SURFsara's Hadoop cluster: [YARN resourcemanager](http://head05.hathi.surfsara.nl:8088/cluster) (note you will need to be authenticated via kerberos on your machine to visit the resourcemanager link) for executors.

First initialize kerberos via a Jupyter terminal. 
In the terminal execute: <BR>
<i>kinit -k -t data/robertop.keytab robertop@CUA.SURFSARA.NL</i><BR>
Print your credentials:


In [1]:
! klist

Ticket cache: FILE:/tmp/krb5cc_1000
Default principal: robertop@CUA.SURFSARA.NL

Valid starting       Expires              Service principal
04/15/2016 15:25:48  04/16/2016 15:25:48  krbtgt/CUA.SURFSARA.NL@CUA.SURFSARA.NL
	renew until 04/15/2016 15:25:48


In [2]:
! hdfs dfs -ls 

Found 5 items
drwx------   - robertop hdfs          0 2016-04-15 12:49 .Trash
drwxr-xr-x   - robertop hdfs          0 2016-04-15 15:32 .sparkStaging
drwx------   - robertop hdfs          0 2016-04-06 15:54 .staging
drwxr-xr-x   - robertop hdfs          0 2016-04-15 14:38 mattia
drwxr-xr-x   - robertop hdfs          0 2016-04-13 10:00 recsys2016Competition


Verify that we can browse HDFS:

Next initialize Spark. Note that the code below starts a job on the Hadoop cluster that will remain running while the notebook is active. Please close and halt the notebook when you are done. Starting the SparkContext can take a little longer. You can check the YARN resourcemanager to see the current status/usage of the cluster.

In [3]:
import os
os.environ['PYSPARK_PYTHON'] = '/usr/local/bin/python2.7'

HDFS_PATH = "hdfs://hathi-surfsara"

from pyspark import SparkConf, SparkContext
from pyspark.sql import SQLContext
sconf = SparkConf()

# Master is now yarn-client. The YARN and hadoop config is read from the environment
sconf.setMaster("yarn-client")

# You can control many Spark settings via the SparkConf. This determines the amount of executors on the cluster:
sconf.set("spark.executor.instances", "200")
#sconf.set("spark.executor.memory", "2g")

# UFW (firewall) is active on the VM. We explicitly opened these ports and Spark should not bind to random ports:
sconf.set("spark.driver.port", 51800)
sconf.set("spark.fileserver.port", 51801)
sconf.set("spark.broadcast.port", 51802)
sconf.set("spark.replClassServer.port", 51803)
sconf.set("spark.blockManager.port", 51804)
sconf.set("spark.authenticate", True)
sconf.set("spark.yarn.keytab", "/home/jovyan/work/data/robertop.keytab")
sconf.set("spark.yarn.access.namenodes", HDFS_PATH + ":8020")

sc = SparkContext(conf=sconf)

sqlCtx = SQLContext(sc) 

print sc

# <hr style="clear: both" />

# Now you can run your code

Pick a clustering algorithm (name of the file that provides a classify(x,y [,threshold]) function)

In [4]:
execfile('../spark-scripts/conventions.py')
execfile('../spark-scripts/splitCluster.py')
#execfile('../spark-scripts/utils.py')
execfile('../spark-scripts/eval.py')
execfile('../spark-scripts/implicitPlaylistAlgoFunctions.py')
execfile('../spark-scripts/implicitPlaylistAlgoMain.py')

CLUSTER_ALGO = 'jaccardBase'

execfile('../spark-scripts/' + CLUSTER_ALGO + '.py')


# Reading the conf file

In [5]:
import json
import copy

BASE_PATH = HDFS_PATH + '/user/robertop/mattia'

conf = {}

conf['split'] = {}
conf['split']['reclistSize'] = 100
conf['split']['callParams'] = {}
conf['split']['excludeAlreadyListenedTest'] = True
conf['split']['name'] = 'test'
conf['split']['split'] = conf['split']['name']
conf['split']['minEventsPerUser'] = 5
conf['split']['inputData'] = HDFS_PATH + '/user/robertop/mattia/clusterBase.split/SenzaRipetizioni_1'
#conf['split']['inputData'] = 's3n://contentwise-research-poli/30musicdataset/newFormat/relations/sessions.idomaar'
conf['split']['bucketName'] = BASE_PATH
conf['split']['percUsTr'] = 0.05
conf['split']['ts'] = int(0.75 * (1421745857 - 1390209860) + 1390209860) - 10000
conf['split']['minEventPerSession'] = 5
conf['split']['onlineTrainingLength'] = 5
conf['split']['GTlength'] = 1
conf['split']['minEventPerSessionTraining'] = 10
conf['split']['minEventPerSessionTest'] = 11
conf['split']['mode'] = 'session'
conf['split']['forceSplitCreation'] = False
conf['split']["prop"] = {'reclistSize': conf['split']['reclistSize']}
conf['split']['type'] = list
conf['split']['out'] = HDFS_PATH + '/user/robertop/mattia/clusterBase.split/'
conf['split']['location'] = '30Mdataset/relations/sessions'

conf['evaluation'] = {}
conf['evaluation']['metric'] = {}
conf['evaluation']['metric']['type'] = 'recall'
conf['evaluation']['metric']['prop'] = {}
conf['evaluation']['metric']['prop']['N'] = [1,2,5,10,15,20,25,50,100]
conf['evaluation']['name'] = 'recall@N'

conf['general'] = {}
conf['general']['clientname'] = "clusterBase.split"
conf['general']['bucketName'] = 'head02.hathi.surfsara.nl/user/robertop/mattia'
conf['general']['tracksPath'] = '30Mdataset/entities/tracks.idomaar.gz'

conf['algo'] = {}
conf['algo']['name'] = 'ClusterBase'
conf['algo']['props'] = {}
# ***** EXAMPLE OF CONFIGURATION *****#
conf['algo']['props']["sessionJaccardShrinkage"] = 5
conf['algo']['props']["clusterSimilarityThreshold"] = 0.1
conf['algo']['props']["expDecayFactor"] = 0.7
# ****** END EXAMPLE ****************#




Pick the list of songs ad create clusters


In [6]:
import json
import string

def my_replace_punct(x):
    ret = ""
    for i in x:
        if i == '+':
            ret += ' '
        else:
            ret += i
    return ret

tracksRDD = sc.textFile(BASE_PATH + '/30Mdataset/entities/tracks.idomaar.gz')
tracksRDD = tracksRDD.repartition(2000)
tracksRDD = tracksRDD.map(lambda x: x.split('\t')).map(lambda x: (x[1], json.loads(x[3])['name'].split('/') ) )
tracksRDD = tracksRDD.map(lambda x: (x[0], " ".join( (x[1][0], x[1][2]) ) ))
tracksRDD = tracksRDD.map(lambda x : (x[0], my_replace_punct(x[1])))
tracksRDD = tracksRDD.map(lambda x: (x[0], tokenize_song(x[1]), x[1]))

#tracksRDD.saveAsPickleFile(BASE_PATH + '/30Mdataset/entities/tracksPickle')

tracksIdsRDD = tracksRDD.map(lambda x: (int(x[0]), [x[0]]))
tracksRDD.count()

5675143

In [7]:
sampleRDD = sc.parallelize(tracksRDD.take(50000))
tracksRDD.getNumPartitions()

2000

In [10]:
'''
def append_equals(x, part):
    name = x[1]
    part = part.value
    for song in part:
        if classify(name, song[1]):
            x[2].append(song[0])
    return x

tracksEqualsRDD = sampleRDD
for i in range(2000):   
    partRDD = sc.pickleFile(BASE_PATH + '/30Mdataset/entities/tracksPickle/part-' + str(i).zfill(5))
    print 'Loaded part ' + str(i)
    part = sc.broadcast(partRDD.collect())
    print 'Broadcasted part ' + str(i)
    tracksEqualsRDD = tracksEqualsRDD.map(lambda x: append_equals(x, part))
    print 'Mapped part ' + str(i)
    tracksEqualsRDD.count()
    print 'Counted part ' + str(i)
    
tracksEqualsRDD.take(10)
'''

"\ndef append_equals(x, part):\n    name = x[1]\n    part = part.value\n    for song in part:\n        if classify(name, song[1]):\n            x[2].append(song[0])\n    return x\n\ntracksEqualsRDD = sampleRDD\nfor i in range(2000):   \n    partRDD = sc.pickleFile(BASE_PATH + '/30Mdataset/entities/tracksPickle/part-' + str(i).zfill(5))\n    print 'Loaded part ' + str(i)\n    part = sc.broadcast(partRDD.collect())\n    print 'Broadcasted part ' + str(i)\n    tracksEqualsRDD = tracksEqualsRDD.map(lambda x: append_equals(x, part))\n    print 'Mapped part ' + str(i)\n    tracksEqualsRDD.count()\n    print 'Counted part ' + str(i)\n    \ntracksEqualsRDD.take(10)\n"

In [11]:
#Build an RDD with ('word' -> (id, name))
wordsRDD = tracksRDD.flatMap(lambda x: [(i, (x[0], x[2])) for i in x[1]] )

#Group by 'word' and keep only the ones with more then 1 song
wordsRDD = wordsRDD.groupByKey().mapValues(list).filter(lambda x: len(x[1]) > 1)

#Eliminate words in top 2% of words
nwords = int(wordsRDD.count()*0.02)
top_words = wordsRDD.map(lambda x: (x[0], len(x[1]))).takeOrdered(nwords, lambda x: -x[1])
wordsRDD = wordsRDD.filter(lambda x: x[0] not in top_words)
print top_words

#Compute a cartesian product for each list of songs with a common word
def filtered_cartesian(x):
    equal_couples = set()
    for i in range(len(x[1])):
        a = x[1][i]
        id_a = x[1][i][0]
        name_a = x[1][i][1]
        
        for j in range(i):
            b = x[1][j]
            id_b = x[1][j][0]
            name_b = x[1][j][1]
            if id_a != id_b:
                if classify(name_a, name_b):
                    equal_couples.add((a,b))
                    
    return (x[0], tuple(equal_couples))

coupleRDD = wordsRDD.map(filtered_cartesian).filter(lambda x: len(x[1]) > 1)
coupleRDD.take(3)

[(u'my', 1370932), (u'she', 1288427), (u'top', 1282068), (u'loves', 1280563), (u'zz', 1278419), (u'automobile', 1278007), (u'the', 827464), (u'of', 266877), (u'in', 203871), (u'a', 181352), (u'you', 153147), (u'i', 152655), (u'and', 138962), (u'remix', 131270), (u'feat', 127194), (u'to', 126452), (u'mix', 108042), (u'me', 107216), (u'love', 98275), (u'live', 87973), (u'no', 86895), (u'on', 83337), (u'for', 81665), (u'version', 80591), (u'de', 69466), (u'it', 64689), (u'is', 59191), (u'original', 59105), (u'la', 54029), (u'your', 51694), (u'2', 46317), (u'from', 46296), (u'with', 44717), (u'all', 43249), (u'by', 41016), (u'1', 39994), (u'one', 39430), (u'2f', 39258), (u'radio', 38913), (u'black', 36460), (u'be', 35733), (u'up', 35709), (u'time', 34986), (u'edit', 34474), (u'at', 34225), (u'don27t', 33719), (u'john', 32599), (u'man', 31925), (u'do', 31909), (u'we', 31481), (u'new', 31191), (u'song', 31059), (u'dj', 30714), (u'night', 29949), (u'orchestra', 29824), (u'out', 29538), (u'thi

[(u'mahiaddin',
  (((u'1953340', u'najwa mahiaddin Before'),
    (u'1953344', u'najwa mahiaddin Wonderland')),
   ((u'1953341', u'najwa mahiaddin Floatin%27'),
    (u'1953340', u'najwa mahiaddin Before')),
   ((u'1953341', u'najwa mahiaddin Floatin%27'),
    (u'1953342', u'najwa mahiaddin Lover')),
   ((u'1953340', u'najwa mahiaddin Before'),
    (u'1953342', u'najwa mahiaddin Lover')),
   ((u'1953344', u'najwa mahiaddin Wonderland'),
    (u'1953342', u'najwa mahiaddin Lover')),
   ((u'1953341', u'najwa mahiaddin Floatin%27'),
    (u'1953344', u'najwa mahiaddin Wonderland')))),
 (u'tiken',
  (((u'3035884', u'Tiken Jah Fakoly Sundjata'),
    (u'3035838', u'Tiken Jah Fakoly L%27Afrique attend')),
   ((u'3035803', u'Tiken Jah Fakoly African R%3Fvolution'),
    (u'3035833', u'Tiken Jah Fakoly Justice')),
   ((u'3035845', u'Tiken Jah Fakoly Marley Foly'),
    (u'3035849', u'Tiken Jah Fakoly Nationalit%C3%A9')),
   ((u'3035846', u'Tiken Jah Fakoly Massadje'),
    (u'3035803', u'Tiken Jah Fak

In [9]:
len(top_words)

9110

In [ ]:
def flat_song_couple(x):
    result = []
    for couple in x[1]:
        result.append((int(couple[0][0]), couple) )
        result.append((int(couple[1][0]), couple) )
    return result
    
#Flatmap the list of couples
#flattedCoupleRDD = coupleRDD.flatMap(lambda x: [i for i in x[1]])
#For each couple, for each song, yield song->couple
flattedCoupleRDD = coupleRDD.flatMap(flat_song_couple)

#Group by key (song). Each song has now one cluster
def merge_couples(x, y):
    return list(set(x) | set(y))

songClusterRDD = flattedCoupleRDD.reduceByKey(merge_couples).map(lambda x: (x[0], [i[0] for i in x[1]]))
songClusterRDD.take(3)

In [ ]:
#In this way we obtain a complete RDD with song -> group of songs
def reduce_to_biggest(x, y):
    bigger = x if len(x) > len(y) else y
    result = sorted(bigger)
    return result
 
tracksIdsRDD = tracksRDD.map(lambda x: (int(x[0]), [x[0]]))
unionRDD = songClusterRDD.union(tracksIdsRDD).reduceByKey(reduce_to_biggest)
unionRDD.take(10)

In [ ]:
#Flip the mapping as cluster->song
clusterSongsRDD = unionRDD.map(lambda x: (' '.join(x[1]), x[0])).groupByKey().mapValues(list)
clusterSongsRDD = clusterSongsRDD.zipWithIndex().map(lambda x: (x[1], x[0][1]))
clusterSongsRDD.take(3)

In [ ]:
#Save clustering
clusterSongsRDD.sav(BASE_PATH + '/clusters/' + CLUSTER_ALGO)